In [1]:
import pandas as pd
import json
from langchain import PromptTemplate

CRASS_FTM_main_data_set_df = pd.read_csv("/Users/lujun.li/projects/causallm-github/causalllm/resource/input/causal_discovery/CRASS-data-set/CRASS_FTM_main_data_set.csv", encoding='latin1',sep=";")

In [2]:
CRASS_FTM_main_data_set_df_length= len(CRASS_FTM_main_data_set_df)
CRASS_FTM_main_data_set_df_transposed= CRASS_FTM_main_data_set_df.T
CRASS_FTM_main_data_set_df_transposed

,0,1,2,3,4,5,6,7,8,9,...,264,265,266,267,268,269,270,271,272,273
PCTID,1,2,3,4,5,6,7,8,9,10,...,265,266,267,268,269,270,271,272,273,274
BatchID,1,1,1,1,1,1,1,1,1,1,...,5,5,5,5,5,5,5,5,5,5
Premise,A woman opens a treasure chest.,A police officer calms down a hostage-taker.,A man talks about a lion.,A girl kisses a boy.,A girl kisses a boy.,A woman sees a fire.,A woman sees a fire.,A branch floats in a pond.,A branch floats in a pond.,A bird lands in a forest.,...,A woman does not order Chinese food.,A woman does not notice a man.,A woman does not run 5 miles.,A girl does not play basketball with her frien...,A woman does not play bass guitar.,A woman does not practice her moves.,A woman does not prepare for someone's reception.,A woman does not put pen to paper.,A woman does not realize something.,A woman does not retire from work.
QCC,What would have happened if the woman had not ...,What would have happened if the police officer...,What would have happened if the man had talked...,What would have happened if the girl had slapp...,What would have happened if the girl had kille...,What would have happened if the woman had touc...,What would have happened if the woman had fed ...,What would have happened if a corpse had float...,What would have happened if a branch had sung ...,What would have happened if a plane had landed...,...,What would have happened if she had ordered Ch...,What would have happened if she had noticed a ...,What would have happened if she had run 5 miles?,What would have happened if she had played bas...,What would have happened if she had played bas...,What would have happened if she had practiced ...,What would have happened if she had prepared it?,What would have happened if she had put pen to...,What would have happened if she had realized s...,What would have happened if she had retired fr...
CorrectAnswer,The treasure chest would have remained closed.,The hostages would have remained in danger.,"Without a barrier, the man would have been eaten.",The girl would have been angry.,She would have been liable to prosecution.,She would have been burned.,The fire would have become larger.,The lake would have been a crime scene.,That is not possible.,The plane would have crashed.,...,The woman would have become less hungry.,The woman would have waved at him.,The woman would have gotten exercise.,The girl would have gotten sweaty.,The woman would have made music.,The woman would have become more skilled.,The woman would have put on make-up.,The woman would have moved her right hand.,The woman would have gotten a new idea.,The woman would not have to work anymore.
Answer1,The treasure chest would have been open.,That is not possible.,"Without a barrier, the lion would have been ea...",The girl would have been happy.,That is not possible.,She would not have been burned.,The fire would have been not hungry anymore.,That is not possible.,It would have been the usual branch singing.,Everything would have been fine.,...,The woman would have become very hungry.,The woman would not have waved at him.,The woman would not have gotten exercise.,The girl would not have gotten sweaty.,The woman would not have made music.,The woman would not have become more skilled.,The woman would not have put on make-up.,The woman would not have moved her right hand.,The woman would not have gotten a new idea.,The woman would still have to work.
Answer2,That is not possible.,The hostage-taker would have released the host...,Nothing special would have happened.,That is not possible.,The boy would have been arrested for assault.,That is not possible.,That is not possible.,The corpse would have swum back to the barge.,It would have been a great concert.,The plane would have landed safe and sound.,...,That is not possible.,That is not possible.,That is not possible.,That is not possible.,That is not possible.,That is not possible.,That is not possible.,That is not possible.,That is not possible.,That is not possible.
PossibleAnswer3,NaN,NaN,N

In [3]:
arraycounter = 0
while arraycounter <  20:
    inputtext = str(CRASS_FTM_main_data_set_df_transposed[arraycounter].PCTID) + ". " + CRASS_FTM_main_data_set_df_transposed[arraycounter].Premise + " " + CRASS_FTM_main_data_set_df_transposed[arraycounter].QCC
    
    print (inputtext)
    arraycounter+=1

1. A woman opens a treasure chest. What would have happened if the woman had not opened the treasure chest?
2. A police officer calms down a hostage-taker. What would have happened if the police officer had not calmed the hostage-taker?
3. A man talks about a lion. What would have happened if the man had talked to the lion?
4. A girl kisses a boy. What would have happened if the girl had slapped the boy?
5. A girl kisses a boy. What would have happened if the girl had killed the boy?
6. A woman sees a fire. What would have happened if the woman had touched the fire?
7. A woman sees a fire. What would have happened if the woman had fed the fire?
8. A branch floats in a pond. What would have happened if a corpse had floated in the pond?
9. A branch floats in a pond. What would have happened if a branch had sung in the pond?
10. A bird lands in a forest. What would have happened if a plane had landed in the forest?
11. A bird lands in a forest. What would have happened if a meteor had lan

In [4]:
arraycounter = 0
while arraycounter <  1:
    inputtext = str(CRASS_FTM_main_data_set_df_transposed[arraycounter].PCTID) + ". " + CRASS_FTM_main_data_set_df_transposed[arraycounter].Premise + " " + CRASS_FTM_main_data_set_df_transposed[arraycounter].QCC
    print("Processing request # : " + str(arraycounter))
    hf_inputs = "\"" + CRASS_FTM_main_data_set_df_transposed[arraycounter].Premise + " " + CRASS_FTM_main_data_set_df_transposed[arraycounter].QCC + "\""
    hf_cl1 = "\"" + CRASS_FTM_main_data_set_df_transposed[arraycounter].CorrectAnswer + "\""
    hf_cl2 = "\"" + CRASS_FTM_main_data_set_df_transposed[arraycounter].Answer1 + "\""
    hf_cl3 = "\"" + CRASS_FTM_main_data_set_df_transposed[arraycounter].Answer2 + "\""
    payload = {"inputs": hf_inputs,"parameters": {"candidate_labels": [hf_cl1,hf_cl2,hf_cl3]}}
    print (payload)
    arraycounter+=1

Processing request # : 0
{'inputs': '"A woman opens a treasure chest. What would have happened if the woman had not opened the treasure chest?"', 'parameters': {'candidate_labels': ['"The treasure chest would have remained closed."', '"The treasure chest would have been open."', '"That is not possible."']}}


In [5]:
arraycounter = 0
system_message = "You are a conscientious assistant, and you must answer my questions exactly as my questions require."
instruction = "Please reference the following three examples (including premise, query, possible answers, and correct answer) to answer the final query."
initial_prompt = """
Instruction: {system_message}
Input: 
{INSTRUCTION}
{INPUT}
"""
mixtral_instruct_initial_prompt = """<s>[INST]
{system_message}
{INSTRUCTION}
{INPUT}
[/INST] 
</s>"""

llama2_chat_initial_prompt = """
<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{INSTRUCTION}
{INPUT}
[/INST]
"""

llama3_chat_init_prompt = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_message}
<|eot_id|><|start_header_id|>user<|end_header_id|>
{INSTRUCTION}
{INPUT}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""


few_shot_prompt = """Premise: A feather falls from a skyscraper. 
Query: What would have happened if a computer had fallen from the skyscraper? 
Possible answers: The computer would have remained intact. That is not possible. The computer would have been crushed. 
Correct answer: The computer would have been crushed.

Premise: A lightning hits a tree. 
Query: What would have happened if a marble would have hit the tree? 
Possible answers: It would have burned down. Nothing special would have happened. The tree would have kissed the lightning. 
Correct answer: Nothing special would have happened. 

Premise: A man drinks a beer. 
Query: What would have happened if the man had drunk a rainbow? 
Possible answers: It would have been tasty. It would have been awful. That is not possible. 
Correct answer: That is not possible.

"""



In [6]:
random_rows = CRASS_FTM_main_data_set_df.sample(n=50, replace=False)

for _, row in random_rows.iterrows():
    instruction = "Please reference the following three examples (including premise, query, possible answers, and correct answer) to answer the final query."
    few_shot_prompt = """
    Examples: 

    Premise: {}
    Query: {}
    Possible answers: {}
    Correct answer: {}
    """.format(row.Premise, row.QCC, row.CorrectAnswer, row.Answer1)

    hf_inputs = "Premise: {}\nQuery: {}".format(row.Premise, row.QCC)
    hf_cl1 = row.CorrectAnswer
    hf_cl2 = row.Answer1
    hf_cl3 = row.Answer2
    prompt = few_shot_prompt + hf_inputs + "\nPossible answers: " + hf_cl1 + hf_cl2 + hf_cl3 + "\nCorrect answer:"
    # Add instruction and prompt to dataframe
    random_rows.at[_, 'instruction'] = instruction
    random_rows.at[_, 'input'] = prompt


In [7]:
# CRASS_FTM_main_data_set_df.sample(1).to_dict()['input']

In [8]:
from langchain import PromptTemplate

initial_prompt_template = PromptTemplate.from_template(initial_prompt)
llama2_chat_initial_prompt_template = PromptTemplate.from_template(
    llama2_chat_initial_prompt
)
mixtral_instruct_initial_prompt_template = PromptTemplate.from_template(
    mixtral_instruct_initial_prompt
)
llama3_chat_initial_prompt_template = PromptTemplate.from_template(
    llama3_chat_init_prompt
)

def generate_initial_prompts(row):
    input = row["input"]
    instruction = row["instruction"]
    initial_prompt = initial_prompt_template.format(
        INSTRUCTION=instruction, INPUT=input, system_message=system_message
    )

    llama2_chat_initial_prompt = llama2_chat_initial_prompt_template.format(
        INSTRUCTION=instruction, INPUT=input, system_message=system_message
    )

    llama3_chat_initial_prompt = llama3_chat_initial_prompt_template.format(
        INSTRUCTION=instruction, INPUT=input, system_message=system_message
    )

    mixtral_instruct_initial_prompt = mixtral_instruct_initial_prompt_template.format(
        INSTRUCTION=instruction, INPUT=input, system_message=system_message
    )

    row["initial_prompt"] = initial_prompt
    row["llama2_chat_initial_prompt"] = llama2_chat_initial_prompt
    row["mixtral_instruct_initial_prompt"] = mixtral_instruct_initial_prompt
    row["llama3_chat_initial_prompt"] = llama3_chat_initial_prompt
    row["INSTRUCTION"] = input
    row["INPUT"] = instruction
    return row

import numpy as np
random_rows = random_rows.apply(generate_initial_prompts, axis=1)
temperatures = np.arange(0.1, 2.0, 0.3)
temperature_df = pd.DataFrame({"Temperature": temperatures})
merged_df = temperature_df.merge(random_rows, how="cross")

In [9]:
repetitions = np.arange(0,10,1)
repetitions_df = pd.DataFrame({"Repetition_times": repetitions})
merged_df_rep = repetitions_df.merge(merged_df, how="cross")

In [10]:
merged_df_rep.to_csv("resource/input/causal_discovery/causal_discovery_experiment_prompt.csv")

In [11]:
merged_df_rep.head(1)

,Repetition_times,Temperature,PCTID,BatchID,Premise,QCC,CorrectAnswer,Answer1,Answer2,PossibleAnswer3,canary_string,instruction,input,initial_prompt,llama2_chat_initial_prompt,mixtral_instruct_initial_prompt,llama3_chat_initial_prompt,INSTRUCTION,INPUT
0,0,0.1,162,3,A person loves a product.,What would have happened if the person had hat...,The person would not have bought the product.,The person would have bought the product.,That is not possible.,NaN,NaN,Please reference the following three examples ...,\n Examples: \n\n Premise: A person love...,\nInstruction: You are a conscientious assista...,\n<s>[INST] <<SYS>>\nYou are a conscientious a...,"<s>[INST]\nYou are a conscientious assistant, ...",\n<|begin_of_text|><|start_header_id|>system<|...,\n Examples: \n\n Premise: A person love...,Please reference the following three examples ...


In [42]:
causal_discovery = pd.read_csv("/Users/lujun.li/projects/causallm-github/causalllm/resource/input/causal_discovery/causal_discovery_experiment_prompt.csv")

In [43]:
causal_discovery.shape

(19180, 20)